In [1]:
import pandas as pd
import numpy as np
from decimal import*
import matplotlib.pyplot as plt
from apyori import apriori

In [2]:
df = pd.read_csv("/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/extracted_data.csv", index_col=False).set_index("ID")

In [15]:
df.columns.tolist()

['Project_Name',
 'Bench_File_Name',
 'Bench_Method_Name',
 'itr_to_stds_avrg',
 'itr_to_stds_std',
 'Num_of_change_points_avrg',
 'STD',
 'AVRG',
 'Bench_File_Nloc',
 'Bench_Method_Nloc',
 'Bench_Method_NestingLevel',
 'Bench_Method_CCN',
 'Bench_File_CCN_Avrg',
 'Number_of_tryCatch',
 'Number_of_switchCase',
 'Number_of_systemCalls',
 'Synchronization',
 'Number_of_forLoops',
 'Number_of_whileLoops',
 'Number_of_packages',
 'Number_of_functionCalls',
 'Number_of_parameters',
 'Benchmark_name_length',
 'Number_of_no_steady_forks',
 'class']

# Data Preprocess:

In this part we have done the data cleaning as there were noisy data amon our dataset. There were also some records which their value type was String, although it should have been numeric. The "Synchronization" feature ,which was presenting the occurence of the threading in the code, has been removed as well because it was no longer showing any difference (all the values were zero after doing data cleaning).

In [3]:
df = df.rename(columns={"class":"cls"})

cols = df.columns.tolist()

corrupt = []
for col in cols[3:]:
    corrupt.append(df.index[(df[col] == "Entity: line 83: parser error : Char 0x0 out of allowed range")].tolist())
    corrupt.append(df.index[(df[col] == "Entity: line 87: parser error : Char 0x0 out of allowed range")].tolist())
    corrupt.append(df.index[(df[col] == "Entity: line 137: parser error : Char 0x0 out of allowed range")].tolist())
    corrupt.append(df.index[(df[col] == "Error Parsing: Document is empty")].tolist())

flatten_corrupt = set([j for sub in corrupt for j in sub])
print("Number of cprrupted benchmark records: ", len(flatten_corrupt))

df = df.drop(flatten_corrupt)

#df.iloc[:,3:] = df.iloc[:,3:].astype(float)
for col in cols:
    df[col] = pd.to_numeric(df[col], errors="ignore")

df = df.drop(["Synchronization"], axis=1)

Number of cprrupted benchmark records:  27


In [11]:
df.to_csv("/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/extracted_data_preprocessed.csv")

In [4]:
bench_method_features = ['Bench_Method_Nloc', 'Bench_Method_CCN'
                         , 'Number_of_tryCatch', 'Number_of_switchCase',
                        'Number_of_forLoops', 'Number_of_whileLoops', 'Number_of_functionCalls',
                         'Num_of_change_points_avrg', "itr_to_stds_avrg"]
bench_file_features = ['Bench_File_Nloc', 'Bench_File_CCN_Avrg', 'Number_of_packages'
                         , 'Benchmark_name_length', "Num_of_change_points_avrg",
                      "itr_to_stds_avrg"]

#dfui = df.drop(df.iloc[:, 3:8].columns.tolist(), axis=1).copy()
dfui = df.copy()
projects_names = list(set(dfui["Project_Name"].tolist()))
bench_files_name = list(set(dfui["Bench_File_Name"].tolist()))
dfui

,Project_Name,Bench_File_Name,Bench_Method_Name,itr_to_stds_avrg,itr_to_stds_std,Num_of_change_points_avrg,STD,AVRG,Bench_File_Nloc,Bench_Method_Nloc,...,Number_of_switchCase,Number_of_systemCalls,Number_of_forLoops,Number_of_whileLoops,Number_of_packages,Number_of_functionCalls,Number_of_parameters,Benchmark_name_length,Number_of_no_steady_forks,cls
ID,,,,,,,,,,,,,,,,,,,,,
91f071f1f69bf48297887879cf5e5613552a29bc,protostuff__protostuff,RuntimeSchemaBenchmark.java,runtime_sparse_deserialize_10_int_field,442.0,595.0,3.1,6.510561e-08,1.869715e-07,277.0,6.0,...,0,0,0,0,22,2,0.0,27.0,0.0,1.0
fb2459909444ecbdf72e47a11179359e56778dcd,netty__netty,AsciiStringBenchmark.java,hashCodeBenchBytesOld,0.0,0.0,1.3,6.980091e-09,1.831741e-07,55.0,8.0,...,0,0,1,0,12,4,1.0,25.0,0.0,1.0
1c51969ca9fc7c1b5a7ac533bfa76899ac30ed88,apache__hive,VectorGroupByOperatorBench.java,testAggCount,4.0,11.0,1.8,1.043777e-08,8.916153e-08,177.0,3.0,...,0,0,0,0,43,1,6.0,31.0,0.0,1.0
2b3a09d375526c8e9c7641660de34577306fb84e,apache__logging-log4j2,ThreadsafeDateFormatBenchmark.java,fastDateFormat,520.0,712.0,7.5,2.670392e-07,6.533268e-07,183.0,3.0,...,0,1,0,0,14,2,0.0,34.0,2.0,0.0
ca5d32819509b08c929db359c6f8dddad84e62f4,RoaringBitmap__RoaringBitmap,BatchIteratorBenchmark.java,batchIterate,322.0,2.0,3.5,1.180394e-04,1.245521e-03,54.0,11.0,...,0,0,1,1,3,3,4.0,27.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1a3b2d7b3a6ece03ec9db59ba9385a7199cace1b,apache__tinkerpop,GremlinGroovyScriptEngineBenchmark.java,testEvalLongCached,126.0,155.0,2.7,1.093954e-02,5.990179e-04,79.0,3.0,...,0,0,0,0,18,1,0.0,39.0,0.0,1.0
0b0eb41a2e416dd93cf156632edab78e7227fab6,eclipse__jetty.project,StringReplaceBenchmark.java,testJavaStringReplaceReduce,683.0,912.0,5.3,1.698672e-07,2.915286e-07,85.0,4.0,...,0,0,0,0,15,2,2.0,27.0,0.0,1.0
8ac7776547858abdceb1e0b055d762870adcc23a,imglib__imglib2,SyncedPositionablesBenchmark.java,benchmark5,582.0,0.0,3.0,1.060187e-02,3.083067e-02,64.0,4.0,...,0,0,0,0,16,12,0.0,33.0,0.0,1.0


# Data Mining:

### File Scope
comparing benchFiles in which more than half of its benchMethods are no_steady (benchMethod with at lesat one no_steady fork) vs the benchFiles that all of its benchMethods are steady (bench method without any no_stedy fork).

In [72]:
stds_nstds_feature_diff = list()
no_steady_benchFiles_names = list()
inconsistence_benchFiles_names = list()
steady_benchFiles_names = list()
benchFile_names = list(set(dfui["Bench_File_Name"].tolist()))
for benchfile in benchFile_names:
    set1 = dfui[(dfui["Bench_File_Name"]== benchfile) & (dfui["Number_of_no_steady_forks"] >=1)]
    #print(len(set1))
    set2 = dfui[(dfui["Bench_File_Name"]== benchfile)]
    #print(len(set2))
    #print("........................................")
    if len(set1)>= (len(set2)/2):
        no_steady_benchFiles_names.append(benchfile)
    elif len(set1) == 0:
        steady_benchFiles_names.append(benchfile)
    else:
        inconsistence_benchFiles_names.append(benchfile)

print("no_steady benchfiles: ", len(no_steady_benchFiles_names))
no_steady_benchFiles = dfui.loc[dfui['Bench_File_Name'].isin(no_steady_benchFiles_names)][bench_file_features]
print("steady benchfiles: ", len(steady_benchFiles_names))
steady_benchFiles = dfui.loc[dfui['Bench_File_Name'].isin(steady_benchFiles_names)][bench_file_features]
print("inconsistence_benchFiles: ", len(inconsistence_benchFiles_names))
inconsistence_benchFiles = dfui.loc[dfui['Bench_File_Name'].isin(inconsistence_benchFiles_names)][bench_file_features]

x = no_steady_benchFiles.mean().tolist()
x = np.array(x, dtype=float)
y = steady_benchFiles.mean().tolist()
y = np.array(y, dtype=float)
stds_nstds_feature_diff.append( np.subtract(x, y) )
y = inconsistence_benchFiles.mean().tolist()
y = np.array(y, dtype=float)
stds_nstds_feature_diff.append( np.subtract(x, y) )

tmp = pd.DataFrame(stds_nstds_feature_diff)
tmp.columns = bench_file_features
tmp

no_steady benchfiles:  94
steady benchfiles:  96
inconsistence_benchFiles:  17


,Bench_File_Nloc,Bench_File_CCN_Avrg,Number_of_packages,Benchmark_name_length,Num_of_change_points_avrg,itr_to_stds_avrg
0,7.946071,-0.178082,-2.526579,0.489022,1.180406,227.363123
1,-5.503082,0.250402,-4.955958,0.051168,0.403570,211.673857


### Method Scope
comparing benchMethods which more than half of their forks are no_steady (forks that did not get to steady state in their last 500 iterations) vs the benchMethods that all of their forks are steady (forks that got to steady state in their last 500 iterations).

In [5]:
stds_nstds_feature_diff = list()
max_no_steady = dfui.Number_of_no_steady_forks.max()
no_steady = dfui[(dfui["Number_of_no_steady_forks"] >= max_no_steady)][bench_method_features]
print(len(no_steady))
inconsistence = dfui[(dfui["Number_of_no_steady_forks"] >= 1)
                    & (dfui["Number_of_no_steady_forks"] < max_no_steady)][bench_method_features]
print(len(inconsistence))
steady = dfui[(dfui["Number_of_no_steady_forks"] == 0)][bench_method_features]
print(len(steady))

x = no_steady.mean().tolist()
x = np.array(x, dtype=float)
y = steady.mean().tolist()
y = np.array(y, dtype=float)
stds_nstds_feature_diff.append( np.subtract(x, y) )
x = inconsistence.mean().tolist()
x = np.array(y, dtype=float)
y = steady.mean().tolist()
y = np.array(x, dtype=float)
stds_nstds_feature_diff.append( np.subtract(x, y) )

tmp = pd.DataFrame(stds_nstds_feature_diff)
tmp.columns = bench_method_features
tmp

2
236
321


,Bench_Method_Nloc,Bench_Method_CCN,Number_of_tryCatch,Number_of_switchCase,Number_of_forLoops,Number_of_whileLoops,Number_of_functionCalls,Num_of_change_points_avrg,itr_to_stds_avrg
0,-2.46729,-0.682243,-0.115265,-0.003115,-0.211838,-0.140187,-1.657321,15.927726,-148.714953
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Associatin rule mining by Apriori:

### BMNFC

In [7]:
apri_dataset = dfui[["Number_of_functionCalls", "Number_of_no_steady_forks"]].copy()

Number_of_no_steady_forks = apri_dataset["Number_of_no_steady_forks"].tolist()
for i, record in enumerate(Number_of_no_steady_forks):
    if record == 0:
        Number_of_no_steady_forks[i] = "nsf_zero"
    elif record >=6:
        Number_of_no_steady_forks[i] = "nsf_high"
    else:
        Number_of_no_steady_forks[i] = "nsf_low"
        
functionCalls_mean = apri_dataset["Number_of_functionCalls"].mean()
Number_of_functionCalls = apri_dataset["Number_of_functionCalls"].tolist()
for i, record in enumerate(Number_of_functionCalls):
    if record > functionCalls_mean:
        Number_of_functionCalls[i] = "nfc_high"
    elif record == 0:
        Number_of_functionCalls[i] = "nfc_zero"
    else:
        Number_of_functionCalls[i] = "nfc_average"

apri_dataset["Number_of_functionCalls"] = Number_of_functionCalls
apri_dataset["Number_of_no_steady_forks"] = Number_of_no_steady_forks
apri_dataset_list = apri_dataset.values.tolist()

In [8]:
apri_dataset.to_csv("/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/functionCalls_extracted_data_for_apriori.csv")

In [10]:
rules = apriori(apri_dataset_list, min_lift=1.01, min_length=2)

In [11]:
Results = list(rules)
df_results = pd.DataFrame(Results)
print(df_results["ordered_statistics"].tolist())

[[OrderedStatistic(items_base=frozenset({'nfc_average'}), items_add=frozenset({'nsf_low'}), confidence=0.3989769820971867, lift=1.0470804365836965), OrderedStatistic(items_base=frozenset({'nsf_low'}), items_add=frozenset({'nfc_average'}), confidence=0.7323943661971831, lift=1.0470804365836965)], [OrderedStatistic(items_base=frozenset({'nfc_high'}), items_add=frozenset({'nsf_zero'}), confidence=0.638157894736842, lift=1.111309231021479), OrderedStatistic(items_base=frozenset({'nsf_zero'}), items_add=frozenset({'nfc_high'}), confidence=0.30218068535825543, lift=1.111309231021479)]]


In [12]:
df_results

,items,support,ordered_statistics
0,"(nfc_average, nsf_low)",0.279070,"[((nfc_average), (nsf_low), 0.3989769820971867..."
1,"(nfc_high, nsf_zero)",0.173524,"[((nfc_high), (nsf_zero), 0.638157894736842, 1..."


### BMLOC

In [14]:
apri_dataset = dfui[["Bench_Method_Nloc", "Number_of_no_steady_forks"]].copy()

Number_of_no_steady_forks = apri_dataset["Number_of_no_steady_forks"].tolist()
for i, record in enumerate(Number_of_no_steady_forks):
    if record == 0:
        Number_of_no_steady_forks[i] = "nsf_zero"
    elif record >=6:
        Number_of_no_steady_forks[i] = "nsf_high"
    else:
        Number_of_no_steady_forks[i] = "nsf_low"
        
mloc_mean = apri_dataset["Bench_Method_Nloc"].mean()
Number_of_loc = apri_dataset["Bench_Method_Nloc"].tolist()
for i, record in enumerate(Number_of_loc):
    if record > mloc_mean:
        Number_of_loc[i] = "loc_high"
    else:
        Number_of_loc[i] = "loc_low"

apri_dataset["Bench_Method_Nloc"] = Number_of_loc
apri_dataset["Number_of_no_steady_forks"] = Number_of_no_steady_forks
apri_dataset_list = apri_dataset.values.tolist()

In [15]:
apri_dataset.to_csv("/home/amirmahdi/Documents/Uni/Dr Ezati/Data Challenge Track/Code/benchMethodLOC_extracted_data_for_apriori.csv")

In [16]:
rules = apriori(apri_dataset_list, min_lift=1.01, min_length=2)

In [17]:
Results = list(rules)
df_results = pd.DataFrame(Results)
print(df_results["ordered_statistics"].tolist())

[[OrderedStatistic(items_base=frozenset({'loc_high'}), items_add=frozenset({'nsf_zero'}), confidence=0.6157635467980296, lift=1.0723109740189987), OrderedStatistic(items_base=frozenset({'nsf_zero'}), items_add=frozenset({'loc_high'}), confidence=0.3894080996884735, lift=1.0723109740189984)], [OrderedStatistic(items_base=frozenset({'loc_low'}), items_add=frozenset({'nsf_low'}), confidence=0.40730337078651685, lift=1.0689323205148493), OrderedStatistic(items_base=frozenset({'nsf_low'}), items_add=frozenset({'loc_low'}), confidence=0.6807511737089201, lift=1.0689323205148493)]]


In [18]:
df_results

,items,support,ordered_statistics
0,"(loc_high, nsf_zero)",0.223614,"[((loc_high), (nsf_zero), 0.6157635467980296, ..."
1,"(loc_low, nsf_low)",0.259392,"[((loc_low), (nsf_low), 0.40730337078651685, 1..."


### BFNP

In [128]:
apri_dataset = dfui[["Number_of_packages", "Number_of_no_steady_forks"]].copy()

Number_of_no_steady_forks = apri_dataset["Number_of_no_steady_forks"].tolist()
for i, record in enumerate(Number_of_no_steady_forks):
    if record == 0:
        Number_of_no_steady_forks[i] = "nsf_zero"
    elif record >=6:
        Number_of_no_steady_forks[i] = "nsf_high"
    else:
        Number_of_no_steady_forks[i] = "nsf_low"
        
np_mean = apri_dataset["Number_of_packages"].mean()
Number_of_p = apri_dataset["Number_of_packages"].tolist()
for i, record in enumerate(Number_of_p):
    if record > mloc_mean:
        Number_of_p[i] = "np_high"
    else:
        Number_of_p[i] = "np_low"

apri_dataset["Number_of_packages"] = Number_of_p
apri_dataset["Number_of_no_steady_forks"] = Number_of_no_steady_forks
apri_dataset_list = apri_dataset.values.tolist()

In [129]:
rules = apriori(apri_dataset_list,  min_lift=1.01, min_length=2)

In [130]:
Results = list(rules)
df_results = pd.DataFrame(Results)
#print(df_results["ordered_statistics"].tolist())

In [131]:
df_results

""
